In [1]:
import os
import json
import traceback
import pandas as pd

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
KEY=os.getenv("OPENAI_API_KEY")


In [6]:
# from langchain_openai import ChatOpenAI

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=KEY,  # put key directly here
    model_name="gpt-3.5-turbo",
    temperature=0.5
)


In [7]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000209CE9FC040>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000209CEAD6220>, root_client=<openai.OpenAI object at 0x00000209EA0EAE50>, root_async_client=<openai.AsyncOpenAI object at 0x00000209CE9FCB50>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')